Задание 1: Разработка и тестирование системы управления пользователями

Файл: user_management.py (Классы User и UserManager)

In [1]:
# user_management.py
from typing import List, Optional, Dict

class User:
    """
    Класс для хранения информации о пользователе.
    """
    def __init__(self, name: str, email: str, age: int):
        """
        Инициализирует пользователя.

        :param name: Имя пользователя.
        :param email: Электронная почта пользователя (должна быть уникальной).
        :param age: Возраст пользователя.
        :raises ValueError: Если email некорректен или возраст отрицательный.
        """
        if not name:
            raise ValueError("Имя пользователя не может быть пустым.")
        if "@" not in email or "." not in email.split('@')[-1]: # Простейшая валидация email
            raise ValueError("Некорректный формат email.")
        if not isinstance(age, int) or age < 0:
            raise ValueError("Возраст должен быть неотрицательным целым числом.")

        self.name = name
        self.email = email
        self.age = age

    def __str__(self) -> str:
        return f"Пользователь(Имя: {self.name}, Email: {self.email}, Возраст: {self.age})"

    def __eq__(self, other) -> bool:
        """Пользователи считаются равными, если у них одинаковый email."""
        if isinstance(other, User):
            return self.email == other.email
        return False
    
    def __hash__(self):
        """Для возможности использования User в качестве ключей словаря или элементов множества."""
        return hash(self.email)


class UserManager:
    """
    Класс для управления пользователями: добавление, удаление, поиск.
    """
    def __init__(self):
        self._users: Dict[str, User] = {} # Храним пользователей в словаре: email -> User object

    def add_user(self, user: User) -> bool:
        """
        Добавляет пользователя в систему.

        :param user: Объект пользователя для добавления.
        :return: True, если пользователь успешно добавлен, False, если пользователь с таким email уже существует.
        :raises TypeError: Если переданный объект не является экземпляром User.
        """
        if not isinstance(user, User):
            raise TypeError("Можно добавлять только объекты класса User.")
        
        if user.email in self._users:
            print(f"Ошибка: Пользователь с email '{user.email}' уже существует.")
            return False
        
        self._users[user.email] = user
        print(f"Пользователь '{user.name}' ({user.email}) успешно добавлен.")
        return True

    def remove_user(self, email: str) -> bool:
        """
        Удаляет пользователя из системы по email.

        :param email: Email пользователя для удаления.
        :return: True, если пользователь успешно удален, False, если пользователь с таким email не найден.
        """
        if email in self._users:
            removed_user_name = self._users[email].name
            del self._users[email]
            print(f"Пользователь '{removed_user_name}' ({email}) успешно удален.")
            return True
        else:
            print(f"Ошибка: Пользователь с email '{email}' не найден для удаления.")
            return False

    def find_user_by_email(self, email: str) -> Optional[User]:
        """
        Ищет пользователя по email.

        :param email: Email для поиска.
        :return: Объект пользователя, если найден, иначе None.
        """
        return self._users.get(email)

    def get_all_users(self) -> List[User]:
        """Возвращает список всех пользователей."""
        return list(self._users.values())

Файл: test_user_management.py (Модульные тесты для User и UserManager)

In [5]:
# test_user_management.py
import unittest

class TestUser(unittest.TestCase):
    """Тесты для класса User."""

    def test_user_creation_valid(self):
        """Тест успешного создания пользователя."""
        user = User("Иван Петров", "ivan@example.com", 30)
        self.assertEqual(user.name, "Иван Петров")
        self.assertEqual(user.email, "ivan@example.com")
        self.assertEqual(user.age, 30)

    def test_user_creation_invalid_email(self):
        """Тест создания пользователя с невалидным email."""
        with self.assertRaisesRegex(ValueError, "Некорректный формат email."):
            User("Тест", "test_invalid_email", 25)
        with self.assertRaisesRegex(ValueError, "Некорректный формат email."):
            User("Тест", "test@invalid", 25)

    def test_user_creation_invalid_age(self):
        """Тест создания пользователя с невалидным возрастом."""
        with self.assertRaisesRegex(ValueError, "Возраст должен быть неотрицательным целым числом."):
            User("Тест", "test@example.com", -5)
        with self.assertRaisesRegex(ValueError, "Возраст должен быть неотрицательным целым числом."):
            User("Тест", "test@example.com", "двадцать") # type: ignore

    def test_user_creation_empty_name(self):
        """Тест создания пользователя с пустым именем."""
        with self.assertRaisesRegex(ValueError, "Имя пользователя не может быть пустым."):
            User("", "test@example.com", 25)
            
    def test_user_equality(self):
        """Тест сравнения пользователей (по email)."""
        user1 = User("Алиса", "alice@example.com", 28)
        user2 = User("Алиса Смит", "alice@example.com", 30) # Другое имя и возраст, но тот же email
        user3 = User("Боб", "bob@example.com", 35)
        
        self.assertEqual(user1, user2)
        self.assertNotEqual(user1, user3)
        self.assertNotEqual(user1, "alice@example.com") # Сравнение с другим типом


class TestUserManager(unittest.TestCase):
    """Тесты для класса UserManager."""

    def setUp(self):
        """Настройка перед каждым тестом: создаем экземпляр UserManager."""
        self.manager = UserManager()
        self.user1 = User("Иван Сидоров", "ivan.sidorov@example.com", 25)
        self.user2 = User("Мария Кузнецова", "maria.k@example.com", 32)
        self.user_duplicate_email = User("Петр Иванов", "ivan.sidorov@example.com", 40)

    def test_add_user_successful(self):
        """Тест успешного добавления нового пользователя."""
        self.assertTrue(self.manager.add_user(self.user1))
        self.assertIn(self.user1.email, self.manager._users) # Проверяем внутреннее хранилище
        self.assertEqual(self.manager.find_user_by_email(self.user1.email), self.user1)

    def test_add_user_duplicate_email(self):
        """Тест попытки добавления пользователя с уже существующим email."""
        self.manager.add_user(self.user1)
        self.assertFalse(self.manager.add_user(self.user_duplicate_email))
        # Убедимся, что пользователь не был перезаписан
        original_user_in_manager = self.manager.find_user_by_email(self.user1.email)
        self.assertIsNotNone(original_user_in_manager)
        if original_user_in_manager: # Доп. проверка для mypy
            self.assertEqual(original_user_in_manager.name, "Иван Сидоров") 

    def test_add_user_invalid_type(self):
        """Тест попытки добавления объекта, не являющегося User."""
        with self.assertRaisesRegex(TypeError, "Можно добавлять только объекты класса User."):
            self.manager.add_user("не пользователь") # type: ignore

    def test_remove_user_existing(self):
        """Тест удаления существующего пользователя."""
        self.manager.add_user(self.user1)
        self.assertTrue(self.manager.remove_user(self.user1.email))
        self.assertIsNone(self.manager.find_user_by_email(self.user1.email))
        self.assertNotIn(self.user1.email, self.manager._users)

    def test_remove_user_non_existing(self):
        """Тест попытки удаления несуществующего пользователя."""
        self.assertFalse(self.manager.remove_user("non.existent@example.com"))

    def test_find_user_by_email_existing(self):
        """Тест поиска существующего пользователя по email."""
        self.manager.add_user(self.user1)
        self.manager.add_user(self.user2)
        found_user = self.manager.find_user_by_email(self.user1.email)
        self.assertEqual(found_user, self.user1)

    def test_find_user_by_email_non_existing(self):
        """Тест поиска несуществующего пользователя по email."""
        self.assertIsNone(self.manager.find_user_by_email("unknown@example.com"))

    def test_get_all_users(self):
        """Тест получения списка всех пользователей."""
        self.assertEqual(self.manager.get_all_users(), []) # Изначально пуст
        self.manager.add_user(self.user1)
        self.manager.add_user(self.user2)
        all_users = self.manager.get_all_users()
        self.assertEqual(len(all_users), 2)
        self.assertIn(self.user1, all_users)
        self.assertIn(self.user2, all_users)

# if __name__ == '__main__':
#     unittest.main(verbosity=2)

Задание 2: Разработка и тестирование калькулятора статистики

Файл: statistics_calculator.py (Класс StatisticsCalculator)

In [4]:
# statistics_calculator.py
from typing import List, Union, NewType
import math

Number = NewType('Number', Union[int, float]) # Для type hinting

class StatisticsCalculator:
    """
    Класс для вычисления среднего арифметического, медианы и дисперсии
    для списка чисел.
    """

    def mean(self, data: List[Number]) -> Optional[float]:
        """
        Вычисляет среднее арифметическое.

        :param data: Список чисел (int или float).
        :return: Среднее арифметическое или None, если список пуст.
        :raises TypeError: Если в списке есть нечисловые значения.
        """
        if not data:
            return None
        if not all(isinstance(x, (int, float)) for x in data):
            raise TypeError("Все элементы списка должны быть числами (int или float).")
        return sum(data) / len(data)

    def median(self, data: List[Number]) -> Optional[float]:
        """
        Вычисляет медиану.

        :param data: Список чисел (int или float).
        :return: Медиана или None, если список пуст.
        :raises TypeError: Если в списке есть нечисловые значения.
        """
        if not data:
            return None
        if not all(isinstance(x, (int, float)) for x in data):
            raise TypeError("Все элементы списка должны быть числами (int или float).")
        
        sorted_data = sorted(data)
        n = len(sorted_data)
        mid_index = n // 2

        if n % 2 == 1: # Нечетное количество элементов
            return float(sorted_data[mid_index])
        else: # Четное количество элементов
            return (sorted_data[mid_index - 1] + sorted_data[mid_index]) / 2.0

    def variance(self, data: List[Number], is_sample: bool = False) -> Optional[float]:
        """
        Вычисляет дисперсию (несмещенную выборочную или генеральную).

        :param data: Список чисел (int или float).
        :param is_sample: True для выборочной дисперсии (делитель n-1), 
                          False для генеральной дисперсии (делитель n).
        :return: Дисперсия или None, если данных недостаточно (менее 1 для генеральной, менее 2 для выборочной).
        :raises TypeError: Если в списке есть нечисловые значения.
        """
        if not all(isinstance(x, (int, float)) for x in data):
            raise TypeError("Все элементы списка должны быть числами (int или float).")

        n = len(data)
        if n == 0:
            return None
        if is_sample and n < 2: # Для выборочной дисперсии нужно хотя бы 2 элемента
            print("Предупреждение: Для выборочной дисперсии требуется как минимум 2 элемента.")
            return None 
            # Или можно raise ValueError("Для выборочной дисперсии требуется как минимум 2 элемента.")
        
        mean_val = self.mean(data)
        if mean_val is None: # Должно быть обработано проверкой n==0 выше, но для надежности
            return None 
            
        squared_diffs = [(x - mean_val) ** 2 for x in data]
        
        if is_sample:
            return sum(squared_diffs) / (n - 1)
        else: # Генеральная совокупность
            return sum(squared_diffs) / n

Задание 3: Разработка и тестирование файлового менеджера

In [7]:
# file_manager.py
import os
import json
import csv
from typing import Any, List, Dict

class FileManager:
    """
    Класс для выполнения базовых операций с файлами:
    чтение, запись и удаление.
    Поддерживает текстовые файлы, JSON и CSV.
    """

    def write_file(self, filepath: str, data: Any, mode: str = 'text', encoding: str = 'utf-8') -> bool:
        """
        Записывает данные в файл.

        :param filepath: Путь к файлу.
        :param data: Данные для записи.
        :param mode: Режим записи ('text', 'json', 'csv').
        :param encoding: Кодировка файла (по умолчанию utf-8).
        :return: True в случае успеха, False в случае ошибки.
        """
        try:
            # Создаем директории, если они не существуют
            os.makedirs(os.path.dirname(filepath), exist_ok=True)

            if mode == 'text':
                if not isinstance(data, str):
                    raise TypeError("Для текстового режима данные должны быть строкой.")
                with open(filepath, 'w', encoding=encoding) as f:
                    f.write(data)
            elif mode == 'json':
                with open(filepath, 'w', encoding=encoding) as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
            elif mode == 'csv':
                if not isinstance(data, list) or not all(isinstance(row, list) for row in data):
                    raise TypeError("Для CSV режима данные должны быть списком списков (list of lists).")
                with open(filepath, 'w', newline='', encoding=encoding) as f:
                    writer = csv.writer(f)
                    writer.writerows(data) # Записываем все строки
            else:
                print(f"Ошибка: Неподдерживаемый режим записи '{mode}'.")
                return False
            
            print(f"Данные успешно записаны в файл: '{filepath}' (режим: {mode})")
            return True
        except (IOError, TypeError, json.JSONDecodeError) as e: # json.JSONDecodeError здесь лишний для записи, но может быть при dump
            print(f"Ошибка при записи в файл '{filepath}': {e}")
            return False
        except Exception as e:
            print(f"Непредвиденная ошибка при записи в файл '{filepath}': {e}")
            return False


    def read_file(self, filepath: str, mode: str = 'text', encoding: str = 'utf-8') -> Any:
        """
        Читает данные из файла.

        :param filepath: Путь к файлу.
        :param mode: Режим чтения ('text', 'json', 'csv').
        :param encoding: Кодировка файла (по умолчанию utf-8).
        :return: Прочитанные данные или None в случае ошибки или если файл не найден.
        """
        if not os.path.exists(filepath):
            print(f"Ошибка: Файл '{filepath}' не найден.")
            return None
        try:
            if mode == 'text':
                with open(filepath, 'r', encoding=encoding) as f:
                    return f.read()
            elif mode == 'json':
                with open(filepath, 'r', encoding=encoding) as f:
                    return json.load(f)
            elif mode == 'csv':
                data_list = []
                with open(filepath, 'r', newline='', encoding=encoding) as f:
                    reader = csv.reader(f)
                    for row in reader:
                        data_list.append(row)
                return data_list
            else:
                print(f"Ошибка: Неподдерживаемый режим чтения '{mode}'.")
                return None
        except (IOError, json.JSONDecodeError, csv.Error) as e:
            print(f"Ошибка при чтении файла '{filepath}': {e}")
            return None
        except Exception as e:
            print(f"Непредвиденная ошибка при чтении файла '{filepath}': {e}")
            return None

    def delete_file(self, filepath: str) -> bool:
        """
        Удаляет файл.

        :param filepath: Путь к файлу для удаления.
        :return: True в случае успеха, False если файл не найден или произошла ошибка.
        """
        if not os.path.exists(filepath):
            print(f"Ошибка: Файл '{filepath}' не найден для удаления.")
            return False
        try:
            os.remove(filepath)
            print(f"Файл '{filepath}' успешно удален.")
            return True
        except OSError as e: # Более специфичное исключение для файловых операций
            print(f"Ошибка при удалении файла '{filepath}': {e}")
            return False
        except Exception as e:
            print(f"Непредвиденная ошибка при удалении файла '{filepath}': {e}")
            return False


class TestFileManager(unittest.TestCase):
    """Тесты для класса FileManager."""

    def setUp(self):
        """Настройка перед каждым тестом."""
        self.fm = FileManager()
        self.test_dir = "test_files_temp" # Временная директория для тестовых файлов
        os.makedirs(self.test_dir, exist_ok=True)
        
        # Определяем пути к тестовым файлам
        self.txt_filepath = os.path.join(self.test_dir, "test.txt")
        self.json_filepath = os.path.join(self.test_dir, "test.json")
        self.csv_filepath = os.path.join(self.test_dir, "test.csv")
        self.non_existent_filepath = os.path.join(self.test_dir, "non_existent.txt")

    def tearDown(self):
        """Очистка после каждого теста: удаляем созданные файлы и директорию."""
        if os.path.exists(self.txt_filepath):
            os.remove(self.txt_filepath)
        if os.path.exists(self.json_filepath):
            os.remove(self.json_filepath)
        if os.path.exists(self.csv_filepath):
            os.remove(self.csv_filepath)
        if os.path.exists(self.test_dir):
            # Удаляем директорию, только если она пуста (на случай непредвиденных файлов)
            # Для более надежной очистки можно использовать shutil.rmtree, но для тестов этого достаточно
            try:
                os.rmdir(self.test_dir) 
            except OSError:
                print(f"Предупреждение: Не удалось удалить директорию {self.test_dir}, возможно, она не пуста.")


    # --- Тесты для write_file() и read_file() ---
    def test_write_and_read_text_file(self):
        """Тест записи и чтения текстового файла."""
        test_content = "Это тестовая строка.\nИ еще одна строка с русскими буквами."
        self.assertTrue(self.fm.write_file(self.txt_filepath, test_content, mode='text'))
        read_content = self.fm.read_file(self.txt_filepath, mode='text')
        self.assertEqual(read_content, test_content)

    def test_write_text_file_invalid_data_type(self):
        """Тест записи в текстовый файл данных неверного типа."""
        with self.assertRaisesRegex(TypeError, "Для текстового режима данные должны быть строкой."):
             # Моделируем ситуацию, когда write_file вызывает исключение внутри, а не возвращает False
             # Для этого нужно, чтобы FileManager не перехватывал TypeError в write_file для 'text'
             # или чтобы тест проверял возвращаемое значение False и сообщение в stdout (сложнее)
             # Для простоты, здесь мы ожидаем, что TypeError будет поднят
             if not self.fm.write_file(self.txt_filepath, ["список", "не строка"], mode='text'):
                 # Если write_file возвращает False и печатает ошибку, этот тест не пройдет
                 # без изменения логики write_file или способа тестирования.
                 # В текущей реализации FileManager перехватывает TypeError и возвращает False.
                 # Поэтому мы будем проверять возвращаемое значение.
                 pass 
        self.assertFalse(self.fm.write_file(self.txt_filepath, ["список", "не строка"], mode='text'))


    def test_write_and_read_json_file(self):
        """Тест записи и чтения JSON файла."""
        test_data = {"ключ1": "значение1", "число": 123, "список": [1, "два", None]}
        self.assertTrue(self.fm.write_file(self.json_filepath, test_data, mode='json'))
        read_data = self.fm.read_file(self.json_filepath, mode='json')
        self.assertEqual(read_data, test_data)

    def test_write_and_read_csv_file(self):
        """Тест записи и чтения CSV файла."""
        test_data_csv = [
            ["Имя", "Возраст", "Город"],
            ["Алиса", "30", "Москва"],
            ["Борис", "25", "СПб"],
            ["Анна;Иванова", "22", "Киев"] # Пример с разделителем в данных
        ]
        self.assertTrue(self.fm.write_file(self.csv_filepath, test_data_csv, mode='csv'))
        read_data_csv = self.fm.read_file(self.csv_filepath, mode='csv')
        self.assertEqual(read_data_csv, test_data_csv)
    
    def test_write_csv_file_invalid_data_type(self):
        """Тест записи в CSV файл данных неверного типа."""
        self.assertFalse(self.fm.write_file(self.csv_filepath, {"не": "список"}, mode='csv'))
        self.assertFalse(self.fm.write_file(self.csv_filepath, [1, 2, 3], mode='csv')) # не список списков

    def test_write_unsupported_mode(self):
        """Тест записи в неподдерживаемом режиме."""
        self.assertFalse(self.fm.write_file(self.txt_filepath, "data", mode='xml'))

    def test_read_non_existent_file(self):
        """Тест чтения несуществующего файла."""
        self.assertIsNone(self.fm.read_file(self.non_existent_filepath, mode='text'))
        self.assertIsNone(self.fm.read_file(self.non_existent_filepath, mode='json'))
        self.assertIsNone(self.fm.read_file(self.non_existent_filepath, mode='csv'))

    def test_read_unsupported_mode(self):
        """Тест чтения в неподдерживаемом режиме (даже если файл существует)."""
        self.fm.write_file(self.txt_filepath, "test", mode='text') # Создаем файл
        self.assertIsNone(self.fm.read_file(self.txt_filepath, mode='doc'))


    # --- Тесты для delete_file() ---
    def test_delete_existing_file(self):
        """Тест удаления существующего файла."""
        # Создаем файл для удаления
        self.assertTrue(self.fm.write_file(self.txt_filepath, "Для удаления", mode='text'))
        self.assertTrue(os.path.exists(self.txt_filepath)) # Убедимся, что файл создан
        
        self.assertTrue(self.fm.delete_file(self.txt_filepath))
        self.assertFalse(os.path.exists(self.txt_filepath)) # Убедимся, что файл удален

    def test_delete_non_existent_file(self):
        """Тест попытки удаления несуществующего файла."""
        self.assertFalse(self.fm.delete_file(self.non_existent_filepath))
        
    def test_file_operations_in_subdirectory(self):
        """Тест операций с файлами во вложенной директории."""
        subdir = os.path.join(self.test_dir, "subdir")
        filepath_in_subdir = os.path.join(subdir, "file_in_subdir.txt")
        content = "Содержимое файла во вложенной папке."

        # FileManager должен создавать директории при записи
        self.assertTrue(self.fm.write_file(filepath_in_subdir, content, mode='text'))
        self.assertTrue(os.path.exists(filepath_in_subdir))
        
        read_content = self.fm.read_file(filepath_in_subdir, mode='text')
        self.assertEqual(read_content, content)
        
        self.assertTrue(self.fm.delete_file(filepath_in_subdir))
        self.assertFalse(os.path.exists(filepath_in_subdir))
        
        # Очищаем поддиректорию (если она пуста)
        if os.path.exists(subdir):
            try:
                os.rmdir(subdir)
            except OSError:
                 print(f"Предупреждение: Не удалось удалить поддиректорию {subdir}.")


